In [4]:
def web_price_data_scrapping(list_of_dates):
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from bs4 import BeautifulSoup
    import pandas as pd
    import time
    output = pd.DataFrame()
    for date in list_of_dates:
      
        # Set options for Chrome WebDriver
        options = Options()
        options.add_argument('--headless')  # To run in headless mode (without opening a visible browser window)
        options.add_argument('--disable-gpu')
        options.add_argument('--no-sandbox')
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36")
        
        # Initialize Chrome WebDriver without specifying the path
        driver = webdriver.Chrome(options=options)
        
        # Define the URL for the option chain page
        url = "https://fcainfoweb.nic.in/reports/report_menu_web.aspx"
        
        # Open the URL
        driver.get(url)  # open the driver
        time.sleep(2)
        
        
        select_price_report_radio = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "ctl00_MainContent_Rbl_Rpt_type_0"))
        )
        
        time.sleep(1)
        select_price_report_radio.send_keys("Price report")
        time.sleep(2)
        
        select_price_report_type = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID,"ctl00_MainContent_Ddl_Rpt_Option0"))
        )
        time.sleep(1)
        select_price_report_type.send_keys("Daily Prices")
        time.sleep(2)
        
        date_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID,"ctl00_MainContent_Txt_FrmDate"))
        )
        time.sleep(1)
        date_input.clear()
        date_input.send_keys(date)   # "15/12/2023" : date format used by the webssite
        time.sleep(3)
        
        get_data_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "ctl00_MainContent_btn_getdata1"))
        )
        time.sleep(1)
        get_data_button.send_keys("Get Data")
        time.sleep(2)
        
        get_table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "gv0"))
        )
        time.sleep(1)
        table_html = get_table.get_attribute('outerHTML')
        time.sleep(1)
        driver.quit() # close the driver
        
        soup = BeautifulSoup(table_html, 'html.parser')
        table = soup.find('table', {'id': 'gv0'})
        # Extracting table headers
        headers = [header.text.strip() for header in table.find_all('th')]  # th means table headers used in html
        
        rows = []
        for row in table.find_all('tr'):  # tr means table rows used in html
            rows.append([val.text.strip() for val in row.find_all('td')])
            
        df = pd.DataFrame(rows[1:], columns=headers)
        df = df[['States/UTs', 'Rice', 'Wheat', 'Atta (Wheat)', 'Gram Dal',
               'Tur/Arhar Dal', 'Urad Dal', 'Moong Dal', 'Masoor Dal', 'Sugar',
               'Milk @', 'Groundnut Oil (Packed)', 'Mustard Oil (Packed)',
               'Vanaspati (Packed)', 'Soya Oil (Packed)', 'Sunflower Oil (Packed)',
               'Palm Oil (Packed)', 'Gur', 'Tea Loose', 'Salt Pack (Iodised)',
               'Potato', 'Onion', 'Tomato']]
        df["Date"] = date
        output = pd.concat([output,df],axis = 0)
    return output

In [5]:
dates = ["10/12/2023","11/12/2023","12/12/2023","13/12/2023","14/12/2023","15/12/2023"] # range of dates for which data need to extract

In [6]:
out = web_price_data_scrapping(dates)

In [7]:
out

,States/UTs,Rice,Wheat,Atta (Wheat),Gram Dal,Tur/Arhar Dal,Urad Dal,Moong Dal,Masoor Dal,Sugar,...,Soya Oil (Packed),Sunflower Oil (Packed),Palm Oil (Packed),Gur,Tea Loose,Salt Pack (Iodised),Potato,Onion,Tomato,Date
0,Average Price,43.71,31.17,36.62,83.12,157.33,123.49,116.56,94.32,44.32,...,121.05,123.82,98.98,51.39,275.28,23.04,24.15,55.48,39.28,10/12/2023
1,Maximum Price,70,54,67,133,195,173,165,153,55,...,180,207,170,167,559,33,70,84,78,10/12/2023
2,Minimum Price,28,21,28,60,87,85,90,70,37,...,96,107,83,35,175,9,11,32,10,10/12/2023
3,Modal Price,40,28,33,80,160,120,120,90,43,...,112.5,130,100,50,265,28,20,60,40,10/12/2023
4,Andaman and Nicobar,50,,67,95,177,145,127,96,54,...,,135,128,75,325,20,40,75,73,10/12/2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,Telangana,51.4,41.2,44.7,75.2,164.6,123.5,114.7,90,41,...,146,117.1,97.3,55.3,304,20.3,27.4,52.2,27.4,15/12/2023
32,Tripura,38.67,,40.33,85.5,144.5,133.5,109,105,48.67,...,116.67,149.33,101.67,72.33,237,10.67,23.33,58.33,58.33,15/12/2023
33,Uttar Pradesh,37.46,26.64,33.09,78.84,149.65,115.15,111.14,94.76,43.84,...,132.66,159.32,112.1,47.69,243.2,26.86,18.19,51.59,35.43,15/12/2023
34,Uttarakhand,36.67,25.33,34,83,151.33,116,114.33,93,44,...,126.67,149,86,52.67,267.67,29,19,54.67,40.33,15/12/2023
